# Recurrent Neural Networks

<a target="_blank" href="https://colab.research.google.com/github/griverat/Meteo-AI/blob/main/notebooks/4.rnn_datos.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Si usa Google Colab, asegúrese de tener habilitada la GPU para este notebook.**

![gpu_colab](../images/colab_gpu.png)

## Descripción

Este notebook contiene el material a desarrollar durante la sesión de redes neuronales recurrentes. Se presentaran los conceptos básicos del uso de inteligencia artificial capaz de capturar tanto patrones espaciales como temporales en los datos.


## Objetivos

- Entender el concepto de redes neuronales recurrentes
- Comparar el rendimiento de una red neuronal recurrente
- Implementar un modelo de pronóstico de series temporales


In [ ]:
# Solo correr esta celda si se usa google colab
# Quitar el comentario (#) a los comandos que comienzan con !

# !pip install ydata_profiling

In [ ]:
!mkdir data
!wget https://raw.githubusercontent.com/griverat/Meteo-AI/main/notebooks/data/puerto_inca.csv -O data/puerto_inca.csv

---

## Redes con características temporales

Si bien las redes neuronales convolucionales (CNN) son muy buenas para capturar patrones espaciales, no son tan buenas para capturar patrones temporales en los datos. Para capturar patrones temporales, se pueden usar redes neuronales recurrentes (RNN) o redes neuronales convolucionales 1D (CNN1D). Para comenzar, se mostrará cómo usar redes neuronales recurrentes para predecir series temporales.

### Importar librerías

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.polynomial.polynomial as poly
import pandas as pd
import tensorflow as tf
import xarray as xr
from ydata_profiling import ProfileReport

plt.rcParams["font.family"] = "monospace"

In [ ]:
def plot_fit(test_label, test_pred, xlim=(15, 45), ylim=(15, 45)):
    coefs_keras = poly.polyfit(test_label.values.flatten(), test_pred, 1)
    ffit_keras = poly.Polynomial(coefs_keras)
    ffit_keras

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.scatter(test_label.values, test_pred, s=5)
    ax.set_xlabel("True")
    ax.set_ylabel("Pred")

    x = np.linspace(15, 45, 100)
    y = ffit_keras(x)
    ax.plot(
        x,
        y,
        color="red",
        lw=1,
        label=f"y = {coefs_keras[0]:.2f} + {coefs_keras[1]:.2f}x",
    )

    ax.legend()

    ax.set_title("True vs Pred")
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # linea de 45 grados
    ax.plot([15, 45], [15, 45], ls="--", lw=0.5, color="black")

    ax.grid(ls="--", lw=0.5)

## Recurrent Neural Networks

Las redes neuronales recurrentes (RNN) son un tipo de red neuronal que se especializa en capturar patrones temporales en los datos. A diferencia de las CNN, las RNN tienen conexiones recurrentes, lo que significa que la salida de una capa se alimenta de nuevo a la misma capa en el siguiente paso de tiempo. Esto permite que las RNN capturen patrones temporales en los datos.

Para comenzar, vamos a cargar datos de la estacion Puerto Inca ubicada en la provincia de Huanuco, Perú.

In [ ]:
station_data = pd.read_csv("data/puerto_inca.csv", skiprows=10)

# renombramos las columnas a algo más amigable
station_data.columns = [
    "date",
    "hour",
    "temp",
    "precip",
    "humidity",
    "wind_dir",
    "wind_speed",
]

# combinamos las columnas de fecha y hora en una sola
station_data["date"] = pd.to_datetime(station_data["date"] + " " + station_data["hour"])
station_data = station_data.drop(columns=["hour"])

# para este ejemplo, solo nos interesa la fecha, temperatura y precipitación
station_data = station_data[["date", "temp", "precip"]]
station_data["temp"] = pd.to_numeric(
    station_data["temp"], errors="coerce", downcast="float"
)
station_data["precip"] = pd.to_numeric(
    station_data["precip"], errors="coerce", downcast="float"
)

# eliminamos las filas con valores faltantes
station_data = station_data.dropna()
station_data = station_data.set_index("date")
station_data.head()

Exploramos los datos

In [ ]:
station_data.describe(exclude=[np.datetime64])

In [ ]:
ProfileReport(station_data)

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
station_data.plot(y="temp", ax=ax[0])
ax[0].set_title("Temperatura")

station_data.plot(y="precip", ax=ax[1])
ax[1].set_title("Precipitación")

Ahora que los datos estan listo, usaremos Keras para crear nuestro modelo de RNN. Primero, guardamos una copia de nuestros datos originales para poder reutilizarlos más adelante.

In [ ]:
station_data_orig = station_data.copy(deep=True)

### Many-to-One RNN

En este caso, vamos a usar una red neuronal recurrente (RNN) para predecir la temperatura de la siguiente hora usando los datos de las ultimas 6h. Este es un ejemplo de una arquitectura de red neuronal recurrente de varios a uno.

Preparamos los datos para el modelo

In [ ]:
station_data["temp_1h"] = station_data["temp"].shift(1)
station_data["temp_2h"] = station_data["temp"].shift(2)
station_data["temp_3h"] = station_data["temp"].shift(3)
station_data["temp_4h"] = station_data["temp"].shift(4)
station_data["temp_5h"] = station_data["temp"].shift(5)

station_data = station_data.dropna()

station_data.head(6)

In [ ]:
station_data["next_temp"] = station_data["temp"].shift(-1)

# eliminamos la última fila, ya que no tiene un valor para la siguiente temperatura
station_data = station_data.dropna()
station_data.head()

Separamos los datos en conjuntos de entrenamiento y prueba

In [ ]:
train_size = int(len(station_data) * 0.85)
# el orden va desde la más antigua a la más reciente
input_vars = ["temp_5h", "temp_4h", "temp_3h", "temp_2h", "temp_1h", "temp"]
output_vars = ["next_temp"]
train_data, test_data = (
    station_data[input_vars].iloc[:train_size],
    station_data[output_vars].iloc[train_size:],
)

train_label, test_label = (
    station_data[input_vars].iloc[:train_size],
    station_data[output_vars].iloc[train_size:],
)

# mean = train_data["temp"].mean()
# std = train_data["temp"].std()

# train_data = (train_data - mean) / std
# test_data = (test_data - mean) / std

Ahora definimos el modelo de RNN usando Keras.

In [ ]:
many_to_one_rnn = tf.keras.models.Sequential(
    [
        tf.keras.layers.SimpleRNN(8, input_shape=(6, 1), return_sequences=True),
        tf.keras.layers.Dense(1),
    ]
)

many_to_one_rnn.compile(optimizer="adam", loss="mean_squared_error")
many_to_one_rnn.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True, start_from_epoch=10
    )
]

history = many_to_one_rnn.fit(
    train_data.values.reshape(-1, 6, 1),
    train_label.values,
    epochs=200,
    batch_size=32,
    validation_split=0.1,
    callbacks=callbacks,
)

In [ ]:
mse = many_to_one_rnn.evaluate(test_data.values.reshape(-1, 1, 1), test_label.values)
print(f"Mean Squared Error: {mse:.2f}")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].plot(history.history["loss"], label="train")
axs[0].plot(history.history["val_loss"], label="validation")
axs[0].set_title("Loss")

test_pred = many_to_one_rnn.predict(test_data.values.reshape(-1, 1)).flatten()
axs[1].plot(test_label.values, label="test")
axs[1].plot(test_pred, label="test prediction")
axs[1].set_title("Prediction")
axs[1].legend()

In [ ]:
plot_fit(test_label, test_pred)

Como podemos ver, tan solo definir un modelo simple de RNN con datos de entrada de 6 horas y una capa densa de salida nos da unos resultados bastante buenos. Pese a que no le hemos dado informacion al modelo de la estacionalidad de los datos, este ha sido capaz de capturar los patrones temporales en los datos y hacer una buena prediccion de la temperatura.

Podemos probar con LSTM y GRU para ver si obtenemos mejores resultados.

In [ ]:
many_to_one_lstm = tf.keras.models.Sequential(
    [
        tf.keras.layers.LSTM(8, input_shape=(6, 1), return_sequences=True),
        tf.keras.layers.Dense(1),
    ]
)

many_to_one_lstm.compile(optimizer="adam", loss="mean_squared_error")
many_to_one_lstm.summary()

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True, start_from_epoch=10
    )
]

history = many_to_one_lstm.fit(
    train_data.values.reshape(-1, 6, 1),
    train_label.values,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=callbacks,
)

In [ ]:
mse = many_to_one_lstm.evaluate(test_data.values.reshape(-1, 1, 1), test_label.values)
print(f"Mean Squared Error: {mse:.2f}")

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].plot(history.history["loss"], label="train")
axs[0].plot(history.history["val_loss"], label="validation")
axs[0].set_title("Loss")

test_pred = many_to_one_lstm.predict(test_data.values.reshape(-1, 1)).flatten()
axs[1].plot(test_label.values, label="test")
axs[1].plot(test_pred, label="test prediction")
axs[1].set_title("Prediction")
axs[1].legend()

In [ ]:
plot_fit(test_label, test_pred)

## Ejericio
La capa Gate Recurrent Unit (GRU) es una variante de la capa LSTM que es más simple y más rápida de entrenar. La GRU tiene menos parámetros que la LSTM y, en general, se comporta de manera similar. En este ejercicio, vamos a probar con una red GRU y comparar los resultados con los obtenidos con la red LSTM y RNN.

Para usar la capa GRU en Keras, simplemente reemplace `LSTM` por `GRU` en la definición de la capa recurrente.

```python
model.add(tf.keras.layers.GRU(units=8, input_shape=(6, 1)))
```

1. Prueba con una red GRU y compara los resultados con los obtenidos con la red LSTM y RNN.

2. Intente cambiar la cantidad de unidades en la capa recurrente y vea cómo afecta el rendimiento del modelo.

3. Hemos estado usando `return_sequences=True` en la capa recurrente. ¿Qué sucede si cambiamos esto a `False`? ¿A qué se debe el cambio en el rendimiento del modelo?

In [ ]:
# code here

### Many-to-Many RNN

Ahora vamos a probar una arquitectura de red neuronal recurrente de varios a varios. En este caso, vamos a usar una red neuronal recurrente (RNN) para predecir la temperatura de las siguientes 6 horas usando los datos de las ultimas 6h.

Vamos a hacer que la red neuronal pronostique los siguientes 6 datos de temperatura al mismo tiempo.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/multistep_lstm.png?raw=1"></img>

Preparamos los datos para el modelo

In [ ]:
station_data_many = station_data.copy(deep=True)
station_data_many

In [ ]:
station_data_many["next_temp_2h"] = station_data_many["temp"].shift(-2)
station_data_many["next_temp_3h"] = station_data_many["temp"].shift(-3)
station_data_many["next_temp_4h"] = station_data_many["temp"].shift(-4)
station_data_many["next_temp_5h"] = station_data_many["temp"].shift(-5)
station_data_many["next_temp_6h"] = station_data_many["temp"].shift(-6)

station_data_many = station_data_many.dropna()
station_data_many.head(12)

In [ ]:
train_size = int(len(station_data_many) * 0.85)
# el orden va desde la más antigua a la más reciente
input_vars = ["temp_5h", "temp_4h", "temp_3h", "temp_2h", "temp_1h", "temp"]
output_vars = [
    "next_temp",
    "next_temp_2h",
    "next_temp_3h",
    "next_temp_4h",
    "next_temp_5h",
    "next_temp_6h",
]
train_data, test_data = (
    station_data_many[input_vars].iloc[:train_size],
    station_data_many[output_vars].iloc[train_size:],
)

train_label, test_label = (
    station_data_many[input_vars].iloc[:train_size],
    station_data_many[output_vars].iloc[train_size:],
)

Ahora definimos el modelo usando Keras.

In [ ]:
many_to_many = tf.keras.models.Sequential(
    [
        tf.keras.layers.LSTM(8, input_shape=(6, 1)),
        tf.keras.layers.Dense(6),
    ]
)

many_to_many.compile(optimizer="adam", loss="mean_squared_error")
many_to_many.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True, start_from_epoch=10
    )
]

history = many_to_many.fit(
    train_data.values.reshape(-1, 6, 1),
    train_label.values.reshape(-1, 6, 1),
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=callbacks,
)

In [ ]:
mse = many_to_many.evaluate(
    test_data.values.reshape(-1, 6, 1), test_label.values.reshape(-1, 6, 1)
)
print(f"Mean Squared Error: {mse:.2f}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.plot(history.history["loss"], label="train")
ax.plot(history.history["val_loss"], label="validation")
ax.set_title("Loss")

Tenemos en total 6 salidas, una para cada paso de tiempo en la secuencia de entrada. Cada salida es una predicción de la temperatura para ese paso de tiempo.

In [ ]:
many_to_many_preds = many_to_many.predict(test_data.values.reshape(-1, 6, 1))
many_to_many_preds = many_to_many_preds.reshape(-1, 6)
many_to_many_preds = pd.DataFrame(many_to_many_preds, columns=output_vars)
many_to_many_preds.set_index(test_label.index, inplace=True)
many_to_many_preds.head()

Ahora vamos a comparar los resultados obtenidos hora a hora con los datos reales.

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 5))
for i, ax in enumerate(axs.flatten()):
    ax.plot(test_label.iloc[:, i], label="true")
    ax.plot(many_to_many_preds.iloc[:, i], label=f"predicción {i+1}h", ls="--")
    rmse = np.sqrt(
        np.mean((test_label.iloc[:, i] - many_to_many_preds.iloc[:, i]) ** 2)
    )
    ax.set_title(f"{output_vars[i]} RMSE: {rmse:.2f}")
    ax.legend()
fig.tight_layout()